In [ ]:
from  datasets import load_dataset
dataset = load_dataset("stanfordnlp/imdb")
train_data = dataset["train"]
test_data = dataset["test"]

# train_data[0]
print(test_data[0])

In [ ]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()  # Load variables from .env file
api_key = os.getenv("API_KEY")

movie_title = "Top Gun: Maverick"

# Search for movie ID
search_url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={movie_title}"
response = requests.get(search_url).json()
movie_id = response["results"][0]["id"]

# Get cast and crew
credits_url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={api_key}"
credits = requests.get(credits_url).json()

# Extract names
actors = [member["name"] for member in credits["cast"][:5]]
directors = [member["name"] for member in credits["crew"] if member["job"] == "Director"]

print("Actors:", actors)
print("Director:", directors)


In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
sample_review = train_data[0]["text"]
doc = nlp(sample_review)

# Extract named entities
entities = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ in ["PERSON"]]
print("Named Entities:", entities)